# 概要
ここでは、GoogleColaboratoryのGPUとGoogleDriveを使用して、BERTのチュートリアルを動かします。  
BERTのチュートリアルをでは、`MRPC`という文章の等価性を評価するタスクを解いていますので、ここでもこれを解いていきます。  
  
GoogleDriveに学習済みモデルがデータセットを保存するため、`500MB`の空きが必要です。

## 全体の流れ

1. Colaboratory notebookの設定
1. GoogleDriveをマウント
1. データセット/BERT学習済みモデルをGoogleDriveへ保存
1. BERTのリポジトリをClone
1. trainデータを使用した、学習済みモデルのfine-tuning
1. testデータを使用した、データ予測
1. 予測結果(出力)の確認

### 1.Colaboratory notebookの設定
当notebookをGPUで実行できるように設定を行います。  
Qiitaの記事を参考にすることをおすすめします。  
1. 上の`ランタイム` -> `ランタイムのタイプを変更`を押す
1. `ノートブックの設定`にて、`ハードウェアのアクセラレータ`を`GPU`に変更
1. 右上の`接続`を押し、`RAM/ディスク`になるまで待つ

### 2.GoogleDriveをマウント

In [ ]:
from google.colab import drive 


drive.mount('/content/drive')

In [ ]:
# 作業ディレクトリをDrive上に作成
!mkdir drive/'My Drive'/bert

In [ ]:
# 作業ディレクトリへ移動
cd drive/'My Drive'/bert

### 3.データセット/BERT学習済みモデルをGoogleDriveへ保存

#### データセットの保存
GLUEデータセットのMRPCタスクのみ保存を行います。

In [ ]:
# DLスクリプトの取得
!git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git

In [ ]:
!python 60c2bdb54d156a41194446737ce03e2e/download_glue_data.py \
--data_dir glue_data --tasks MRPC

#### BERT学習済みモデルの保存
公式が公開している`BERT-Base, Uncasedモデル`を保存します。

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip && \
unzip uncased_L-12_H-768_A-12.zip && \
rm uncased_L-12_H-768_A-12.zip

### TensorFlowのバージョン変更

In [ ]:
%tensorflow_version 1.x

### BERTのリポジトリをClone
BERTリポジトリ内の`run_classifier.py`を実行してタスクを解くため、リポジトリをCloneします。

In [ ]:
!git clone https://github.com/google-research/bert.git

## trainデータを使用した、学習済みモデルのfine-tuning
glue_data/MRPC/train.tsvを使用し、学習済みモデルのfine-tuningを行います。

##### パラメーターの説明
- task_name: 解くタスクを指定
 - MRPC: 2つのニュース記事の等価性を判定
 - CoLa: 文章が文法的に正しいか判定
 - MNLI: テキスト同士の関連性を判定
 - XNLI: 翻訳系のタスク(?)
- do_train: 学習するか否か
- do_eval: 評価結果を表示するか否か
- data_dir: データセット(MRPC)のディレクトリPath
- vocab_file: 学習済みモデルのvocab.txtのPath
- bert_config_file: 学習済みモデルのbert_config.jsonのPath
- init_checkpoint: 学習済みモデルのbert_model.ckpt.***のPath
- max_seq_length: 最長トークンの長さ
- train_batch_size: 学習Batchサイズ
- learning_rate: 学習率
- num_train_epochs: エポック数
- output_dir: 結果出力先

In [ ]:
!python bert/run_classifier.py \
  --task_name=MRPC \
  --do_train=true \
  --do_eval=true \
  --data_dir=glue_data/MRPC \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=tmp/mrpc_output/

完了すると以下の出力がされます。

```
INFO:tensorflow:***** Eval results *****
I0105 10:34:39.428679 140299170789248 run_classifier.py:923] ***** Eval results *****
INFO:tensorflow:  eval_accuracy = 0.85294116
I0105 10:34:39.428822 140299170789248 run_classifier.py:925]   eval_accuracy = 0.85294116
INFO:tensorflow:  eval_loss = 0.4425803
I0105 10:34:40.041571 140299170789248 run_classifier.py:925]   eval_loss = 0.4425803
INFO:tensorflow:  global_step = 343
I0105 10:34:40.041978 140299170789248 run_classifier.py:925]   global_step = 343
INFO:tensorflow:  loss = 0.4425803
I0105 10:34:40.042183 140299170789248 run_classifier.py:925]   loss = 0.4425803
```

### testデータを使用した、データ予測
test.tsvを使用して、予測を行います。

##### パラメーターの説明
- do_predict: 予測を行うか否か
- init_checkpoint: fine-tuning後のbert_model.ckpt.***のPath
- output_dir: 予測結果の出力先

In [ ]:
!python bert/run_classifier.py \
  --task_name=MRPC \
  --do_predict=true \
  --data_dir=glue_data/MRPC \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=tmp/mrpc_output \
  --max_seq_length=128 \
  --output_dir=tmp/mrpc_output_result/

### 予測結果(出力)の確認
GoogleDriveの`bert/tmp/mrpc_output_result/`に、`test_results.tsv`が出力されています。

以上